In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [1]:
from typing import Any

import sympy
import math
import random
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from cirq import GridQubit, ops
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import construct_hamiltonian, get_ground_state_for_g, get_theta_v, get_theta_w, get_theta_r

In [2]:
generator_layers = 2
discriminator_layers = 5
data_bus_size = 3

In [3]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

In [4]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [5]:
pure_gen = gen.copy()
gen.append([disc])

In [6]:
pure_real = real.copy()
real.append([disc])

In [7]:
print("REAL GROUND STATE")
print(pure_real)

REAL GROUND STATE
(0, 2): ───────X───Z───Ry(theta_r)───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │                                   │                                       │
(0, 3): ───────┼─────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────
               │                                                                                                                   │                                       │
(0, 4): ───────┼─────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(

In [8]:
print("GENERATOR")
print(pure_gen)

GENERATOR
(0, 2): ───Rz(l)───Rx(g0)───Rz(g4)───ZZ───────────────Rx(g11)───Rz(g15)───ZZ────────────────
                                     │                                    │
(0, 3): ───Rz(l)───Rx(g1)───Rz(g5)───ZZ^g8───ZZ───────Rx(g12)───Rz(g16)───ZZ^g19───ZZ───────
                                             │                                     │
(0, 4): ───Rz(l)───Rx(g2)───Rz(g6)───ZZ──────ZZ^g10───Rx(g13)───Rz(g17)───ZZ───────ZZ^g21───
                                     │                                    │
(0, 5): ───Rz(l)───Rx(g3)───Rz(g7)───ZZ^g9────────────Rx(g14)───Rz(g18)───ZZ^g20────────────


In [9]:
print("DISCRIMINATOR")
print(disc)

DISCRIMINATOR
(0, 0): ───Rz(l)───Rx(d0)───Rz(d5)───ZZ────────────────Rx(d14)───Rz(d19)───ZZ────────────────Rx(d28)───Rz(d33)───ZZ────────────────Rx(d42)───Rz(d47)───ZZ────────────────Rx(d56)───Rz(d61)───ZZ────────────────
                                     │                                     │                                     │                                     │                                     │
(0, 1): ───Rz(l)───Rx(d1)───Rz(d6)───ZZ^d10───ZZ───────Rx(d15)───Rz(d20)───ZZ^d24───ZZ───────Rx(d29)───Rz(d34)───ZZ^d38───ZZ───────Rx(d43)───Rz(d48)───ZZ^d52───ZZ───────Rx(d57)───Rz(d62)───ZZ^d66───ZZ───────
                                              │                                     │                                     │                                     │                                     │
(0, 2): ───Rz(l)───Rx(d2)───Rz(d7)───ZZ───────ZZ^d12───Rx(d16)───Rz(d21)───ZZ───────ZZ^d26───Rx(d30)───Rz(d35)───ZZ───────ZZ^d40───Rx(d44)───Rz(d49)───ZZ───────ZZ^d54───Rx(d58)───

In [11]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([0] * len(gs)) + \
                   np.random.normal(scale=eps, size=(len(gs),))
init_disc_weights = np.random.normal(size=(len(ds),))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [ ]:
class CustomScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomScheduler, self).__init__()
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return max(math.e ** - ((step+200) / (self.warmup_steps / math.log(100))), 0.01)

In [12]:
learning_rate = CustomScheduler()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [13]:
g_provider = lambda: random.choice([-1, 0])

In [14]:
trainer = Trainer(g_provider, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  use_analytical_expectation=True)

In [15]:
epochs = 101
disc_iteration = 100
gen_iteration = 10
print_interval_epoch = 10
trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration,
      print_interval_epoch=print_interval_epoch)

Instructions for updating:
Use fn_output_signature instead
Epoch 0: discriminator cost = [[0.00379562]]
Prob(real classified as real):  [[0.67620933]]
Prob(fake classified as real):  [[0.68000495]]
Epoch 0: generator cost = [[-0.68000495]]
Prob(fake classified as real):  [[0.68000495]]
Discriminator cost:  [[0.00379562]]
Generator weights: <tf.Variable 'Variable:0' shape=(22,) dtype=float32, numpy=
array([ 0.01764052,  0.00400157,  0.00978738,  0.02240893,  0.01867558,
       -0.00977278,  0.00950088, -0.00151357, -0.00103219,  0.00410599,
        0.00144044,  0.01454274,  0.00761038,  0.00121675,  0.00443863,
        0.00333674,  0.01494079, -0.00205158,  0.00313068, -0.00854096,
       -0.0255299 ,  0.00653619], dtype=float32)>
Discriminator weights <tf.Variable 'Variable:0' shape=(70,) dtype=float32, numpy=
array([  16.247     ,   -2.5803525 ,   26.121424  ,  -46.660286  ,
        105.624084  ,   -3.1000824 ,   29.815437  ,  -40.27243   ,
        -60.96208   ,    8.079186  ,  -47.11

In [16]:
g = 0
size = 3
ground_state = get_ground_state_for_g(g, size)
rad = map_to_radians(g)


trained_disc_weights = tf.Variable(np.array([ 2.86846906e-01,  2.07442139e+02, -1.32505005e+02,  3.37965851e+02,
        1.44702286e+02,  1.67083755e+02,  3.51270142e+02,  4.46218109e+02,
        4.07063354e+02,  1.02569595e+02,  1.62362915e+02,  1.63151154e+02,
        2.18758965e+01, -8.55859451e+01,  1.11934540e+02, -2.94222107e+01,
       -2.17204269e+02,  1.34002686e+02, -1.02429161e+02,  3.34117676e+02,
        1.70663238e-01, -7.37496872e+01,  2.35848450e+02,  5.64170044e+02,
        1.03939911e+02,  1.88091278e+02,  2.74167786e+02,  2.02242035e+02]), dtype=tf.float32)

trained_gen_weights = tf.Variable(np.array([-29.708769 ,  34.885185 ,  47.25734  ,  41.711372 ,  35.44219  ,
        29.7916   , -32.132847 ,   5.453004 ,  94.73532  , -58.75811  ,
       -13.691349 ,  -6.0567594, -25.313757 , -76.478004 , -57.434578 ,
       -71.249245 , -48.34288  ,  10.728431 ,  41.8393   , -22.886314 ,
        -6.871363 , -16.005516, rad]), dtype=tf.float32)




state_vector = tfq.layers.State()(pure_gen, symbol_names=gs + (ls,), symbol_values=tf.reshape(trained_gen_weights, (1, trained_gen_weights.shape[0]))).values.numpy()#%%

traced_out_state = cirq.partial_trace_of_state_vector_as_mixture(state_vector, [0,1,2])

In [17]:
prob, highest_prob_state = max(traced_out_state)
print(f"Max prob state has prob: {prob}")

Max prob state has prob: 0.9834122061729431


In [18]:
highest_prob_state

array([-0.02471295+0.j        , -0.03447054+0.00896384j,
       -0.31766006+0.01939609j, -0.48983648+0.32166958j,
       -0.03840197+0.06002485j,  0.00100844+0.10900943j,
       -0.25798878-0.23478244j, -0.6283039 -0.14148122j], dtype=complex64)

In [19]:
get_ground_state_for_g(g, size)

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
np.arccos(cirq.fidelity(highest_prob_state, get_ground_state_for_g(g,size)))

1.5701855967675986

In [21]:
cirq.fidelity(highest_prob_state, get_ground_state_for_g(g, size))

0.0006107299893317632

In [22]:
trained_disc_weights = tf.Variable(np.array([ 2.86846906e-01,  2.07442139e+02, -1.32505005e+02,  3.37965851e+02,
        1.44702286e+02,  1.67083755e+02,  3.51270142e+02,  4.46218109e+02,
        4.07063354e+02,  1.02569595e+02,  1.62362915e+02,  1.63151154e+02,
        2.18758965e+01, -8.55859451e+01,  1.11934540e+02, -2.94222107e+01,
       -2.17204269e+02,  1.34002686e+02, -1.02429161e+02,  3.34117676e+02,
        1.70663238e-01, -7.37496872e+01,  2.35848450e+02,  5.64170044e+02,
        1.03939911e+02,  1.88091278e+02,  2.74167786e+02,  2.02242035e+02]), dtype=tf.float32)

trained_disc_weights = tf.Variable(np.array([ 
        2.86846906e-01, 0, 0, 0, 0,  
        1.67083755e+02,  3.51270142,  4.46218109, 4.07063354,  1.02569595,  
        -1.362915,  1.63151154, 2.18758965, -8.55859451,  
        1.11934540e+02, -2.9422210, -2.1720426,  134.34002686, -1.0242916,  
        3.34117676e+02, 231.7066323, -7.3749687,  -2.3584845,  5.64170044, 
        -11.03939911e+02, -1.88091278,  0,  2.0224203]), dtype=tf.float32)

trained_gen_weights = tf.Variable(np.array([ 0.01764052,  0.00400157,  0.00978738,  0.02240893,  0.01867558,
       -0.00977278,  0.00950088, -0.00151357, -0.00103219,  0.00410599,
        0.00144044,  0.01454274,  0.00761038,  0.00121675,  0.00443863,
        0.00333674,  0.01494079, -0.00205158,  0.00313068, -0.00854096,
       -0.0255299 ,  0.00653619]), dtype=tf.float32)